In [1]:
import models_predictors
import unet_tf

import cv2
import matplotlib.pyplot as plt 
import numpy as np
from skimage.io import imread, imshow
import scipy.ndimage as ndi

Using TensorFlow backend.


In [2]:
YOLO_MODELS_PATH = "yolo_models/"
TRAIN_PATH = '../../data/raw_data/malaria/transformed_images/'

MASK_CATEGORIES = ['trophozoite','ring', 'schizont', 'gametocyte']
STR_MASK_CATEGORIES = '_'.join(MASK_CATEGORIES)

In [3]:
train_ids = np.load(TRAIN_PATH+'malaria_train_transformed_256_256_names.npy',allow_pickle=True)
test_ids = np.load(TRAIN_PATH+'malaria_test_transformed_256_256_names.npy',allow_pickle=True)

In [4]:
X_test = np.load(TRAIN_PATH+'malaria_test_transformed_256_256.npy',allow_pickle=True)

In [5]:
Y_test =  np.load(TRAIN_PATH+'malaria_test_transformed_256_256_masks_'+STR_MASK_CATEGORIES+'.npy',allow_pickle=True)

In [6]:
model_file = 'YOLOv2-malaria_20190818_trofozoito_esquizonte_gametocito_1024_1024.cfg'
weigths_file = 'YOLOv2-malaria_20190818_trofozoito_esquizonte_gametocito_1024_1024_best.weights'

In [7]:
class_ids, confidences, boxes, images_index, masks = models_predictors.yolo_model_prediction(YOLO_MODELS_PATH, \
                                                                    "obj_es.names", model_file, weigths_file, \
                                                                    test_ids, 1024, 1024, conf_threshold = 0.4)

####### RESULTS ########
Total images tested: 120, Number of detections: 106


In [8]:
model_file2 = 'YOLOv2-malaria_20190818_trofozoito_esquizonte_gametocito_512_512.cfg'
weigths_file2 = 'YOLOv2-malaria_20190818_trofozoito_esquizonte_gametocito_512_512_best.weights'

In [9]:
class_ids2, confidences2, boxes2, images_index2, masks2 = models_predictors.yolo_model_prediction(YOLO_MODELS_PATH, \
                                                                                "obj_es.names", model_file2, \
                                                                                weigths_file2, test_ids, \
                                                                                512, 512, conf_threshold = 0.4)

####### RESULTS ########
Total images tested: 120, Number of detections: 60


In [10]:
nn_name = "u-net_models/unet-malaria_256_256_201908200413_trophozoite_ring_schizont_gametocyte"

In [11]:
u_net = unet_tf.NeuralNetwork()
sess = u_net.load_session_from_file(nn_name, allow_growth=True, same_location=True)

/home/erick/Documents/bioinformatic_thesis_project/code/training/u-net_models/unet-malaria_256_256_201908200413_trophozoite_ring_schizont_gametocyte.meta
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from u-net_models/unet-malaria_256_256_201908200413_trophozoite_ring_schizont_gametocyte
u-net_models/unet-malaria_256_256_201908200413_trophozoite_ring_schizont_gametocyte
u-net_models/unet-malaria_256_256_201908200413_trophozoite_ring_schizont_gametocyte
Parameters of the loaded neural network
	nn_name = u-net_models/unet-malaria_256_256_201908200413_trophozoite_ring_schizont_gametocyte, epoch = 400.11, mb_size = 10
	input_shape = (256, 256, 3), output_shape = (256, 256, 1)
	learn_rate = 0.0000000000, learn_rate_0 = 0.0010000000, dropout_proba = 0.1
	learn_rate_step = 3, learn_rate_pos = 133, learn_rate_alpha = 0.25


In [12]:
test_index = np.arange(0,test_ids.shape[0])

In [13]:
y_test_pred_proba = u_net.get_prediction(sess, X_test)
y_test_pred  = unet_tf.trsf_proba_to_binary(y_test_pred_proba)

In [14]:
test_score, test_metrics= unet_tf.get_score(Y_test, y_test_pred)

/home/erick/anaconda3/envs/bioinformatic_thesis_gpu/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/erick/anaconda3/envs/bioinformatic_thesis_gpu/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/erick/Documents/bioinformatic_thesis_project/code/training/unet_tf.py:766: RuntimeWarning: invalid value encountered in long_scalars
  prec = tp/(tp+fp)
/home/erick/Documents/bioinformatic_thesis_project/code/training/unet_tf.py:782: RuntimeWarning: Mean of empty slice
  score = np.nanmean(summary[:,1]) # Final score.


In [19]:
test_metrics

array([[0.58767773, 0.06666667, 0.2       , 1.        , 3.        ],
       [0.47272042, 0.        , 0.        , 3.        , 5.        ],
       [0.50699245, 0.03333333, 0.1       , 3.        , 9.        ],
       [0.22110553, 0.        , 0.        , 1.        , 1.        ],
       [0.37511231, 0.        , 0.        , 2.        , 5.        ],
       [       nan, 0.        , 0.        , 0.        , 2.        ],
       [0.41262136, 0.        , 0.        , 1.        , 1.        ],
       [0.46814044, 0.        , 0.        , 1.        , 3.        ],
       [0.62424242, 0.3       , 0.1       , 3.        , 1.        ],
       [0.43564356, 0.        , 0.        , 2.        , 2.        ],
       [0.15477214, 0.        , 0.        , 2.        , 2.        ],
       [0.43659286, 0.        , 0.        , 2.        , 6.        ],
       [0.3453617 , 0.        , 0.        , 3.        , 3.        ],
       [0.27123587, 0.        , 0.        , 2.        , 2.        ],
       [0.4568457 , 0.3       , 0.

In [22]:
np.sum(test_metrics[:,3])

284.0

In [23]:
np.sum(test_metrics[:,4])

373.0

In [16]:
y_test_pred.shape

AttributeError: 'list' object has no attribute 'shape'

In [ ]:
fig, axes = plt.subplots(10,5,figsize=(20,20))


for img_id, mask in enumerate(masks[:10]):
    IMG_HEIGHT = 1024
    IMG_WIDTH = 1024
    mask_model1 = mask
    mask_model2 = cv2.resize(masks2[img_id], dsize=(IMG_HEIGHT, IMG_WIDTH), interpolation=cv2.INTER_NEAREST)
    mask_model3 = cv2.resize(np.squeeze(y_test_pred[img_id]), dsize=(IMG_HEIGHT, IMG_WIDTH), interpolation=cv2.INTER_NEAREST)

    overlapping = np.matmul(mask,mask_model2)
    overlapping = np.matmul(mask,mask_model3)
    axes[img_id,0].imshow(mask, cmap='gray')
    axes[img_id,1].imshow(mask_model2, cmap='gray')
    axes[img_id,2].imshow(mask_model3, cmap='gray')
    axes[img_id,3].imshow(overlapping, cmap='gray')
    axes[img_id,4].imshow(np.squeeze(Y_test[img_id]), cmap='gray')